# Analysis

## Data Exploration

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlsxwriter
from IPython.display import Markdown as md
from scipy.sparse import csr_matrix
from sklearn.metrics import silhouette_samples, silhouette_score

%matplotlib inline

import plot_helper

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The data set for this project was downloaded from Kaggle at  
[8anu climbing logbook](https://www.kaggle.com/dcohen21/8anu-climbing-logbook)

First we need to connect to the database provided and extract information into a pandas dataframe.

In [ ]:
# Connect to db
conn = sqlite3.connect('data/8anu.sqlite')

# Read data into pandas
sql = ('SELECT a.id, a.country AS crag_country, a.crag_id, a.crag, a.sector_id, a.sector, a.name AS route, '
       'a.grade_id, g.fra_routes AS grade, a.year, a.date, a.method_id, m.shorthand AS method, a.notes, a.raw_notes, a.rating, '
       'a.user_id, u.country AS user_country, u.city AS user_city, u.sex, u.height, u.weight, u.birth AS birthdate, u.started AS started_climbing '
       'FROM ('
           'SELECT * '
           'FROM ascent '
           'WHERE crag=\'Frankenjura\' AND climb_type=0'
       ') AS a '
       'LEFT OUTER JOIN grade AS g ON a.grade_id=g.id '
       'LEFT OUTER JOIN method AS m ON a.method_id=m.id '
       'LEFT OUTER JOIN user AS u ON a.user_id=u.id '
       )
df = pd.read_sql_query(sql, conn)

# Close db connection
conn.close()

In [ ]:
# Create copy for later user
df_original = df.copy()

The two cells below extract data in CSV, as well as Excel format.

In [ ]:
# Export to .csv for submitting Project Proposal to Udacity
#df.to_csv('data/8anu_franken.csv', sep=';')

In [ ]:
# Export to .xlsx
# Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('data/8anu_franken.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
#df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()

### General information about the data set

Let's take a look at the columns and a few examples we extracted from our data source.

In [ ]:
## Get a list of all columns
df.columns

In [ ]:
df.info()

In [ ]:
df.head(5)

| <p align='left'>`Column` | <p align='left'>Description | <p align='center'> Datatype | <p align='center'> Use | 
|:------|:------|:------:|:------:| 
| <p align='left'>`crag_country` | <p align='left'>3-letter country code of the country where the crag is located. | <p align='center'> categorical | <p align='center'> no | 
| <p align='left'>`crag_id` | <p align='left'>Unique id for identifying each crag. | <p align='center'> int | <p align='center'> no | 
| <p align='left'>`crag` | <p align='left'>Name of the crag | <p align='center'> text | <p align='center'> no | 
| <p align='left'>`sector_id` | <p align='left'>Unique id for identifying each sector. | <p align='center'> int | <p align='center'> yes | 
| <p align='left'>`sector` | <p align='left'>Name of the sector. A sector is a specific area within a crag. | <p align='center'> text | <p align='center'> yes | 
| <p align='left'>`route` | <p align='left'>Name of the route the climber has climbed. | <p align='center'> text | <p align='center'> yes | 
| <p align='left'>`grade_id` | <p align='left'>Unique id for identifying each climbing grade. | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`grade` | <p align='left'>Climbing grade given to that route as per the French grading system | <p align='center'> categorical | <p align='center'> statistics | 
| <p align='left'>`year` | <p align='left'>Year the route was climbed | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`date` | <p align='left'>Date the route was climbed. The date format is number of seconds since 1970-01-01. | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`method_id` | <p align='left'>Unique id for identifying each type of ascent. | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`method` | <p align='left'>The type of ascent the climber made on that route. | <p align='center'> categorical | <p align='center'> statistics | 
| <p align='left'>`notes` | <p align='left'>Additional information the climber provided for this climb, e.g. Soft graded, i.e. fairly easy climb for the given grade | <p align='center'> categorical | <p align='center'> statistics | 
| <p align='left'>`raw_notes` | <p align='left'>Encoding of different notes and combination of notes | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`rating` | <p align='left'>Rating given to the climb by this climber. This is our target attribute. | <p align='center'> int | <p align='center'> target | 
| <p align='left'>`user_id` | <p align='left'>Unique id for this climber. | <p align='center'> int | <p align='center'> yes | 
| <p align='left'>`user_country` | <p align='left'>3-letter country code of the country where this climber is from. | <p align='center'> categorical | <p align='center'> statistics | 
| <p align='left'>`user_city` | <p align='left'>City where this climber is from | <p align='center'> text | <p align='center'> statistics | 
| <p align='left'>`sex` | <p align='left'>The climber's sex. 0 indicates male, 1 indicates female. | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`height` | <p align='left'>The climber's height in cm | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`weight` | <p align='left'>The climber's weight in kg | <p align='center'> int | <p align='center'> statistics | 
| <p align='left'>`birthdate` | <p align='left'>The climber's date of birth | <p align='center'> date | <p align='center'> statistics | 
| <p align='left'>`started_climbing` | <p align='left'>The year the climber started climbing. | <p align='center'> int | <p align='center'> statistics | ![image.png](attachment:image.png)

In the table above the column `Use` indicated how we plan to use the given column.
- `target` - this is a target attribute.
- `yes` - this column will be used during exploration and analysis.
- `no` - not planned to use that column during analysis and exploration.
- `statistics` - this column will not be used for analysis but may be interesting later on to do user statistics etc.

How big is our data set? How many climbs contain a rating?

In [ ]:
records_count = df.shape[0]
ratings_count = df[ df['rating'] == 0].shape[0]


f'There are {records_count:,} data entries in our dataset.'
f'{ratings_count:,} out of these contain a rating. That is approx. {ratings_count / records_count * 100:.2f}%'

We will not use all the columns from the data set. Let's drop unnecessary columns for analysis.

In [ ]:
df = df.drop(columns=['crag_country',
                      'crag_id',
                      'crag',
                      'grade_id',
                      'grade',
                      'year',
                      'date',
                      'method_id',
                      'method',
                      'notes',
                      'raw_notes',
                      'user_country',
                      'user_city',
                      'sex',
                      'height',
                      'weight',
                      'birthdate',
                      'started_climbing'])

### Missing values

Let us take a look if we have **missing values** or **zero values**.

Number of missing values per column:

In [ ]:
df.isnull().sum()

This means there are no entries missing in the columns that we look at.

Number of zero values per column:

In [ ]:
(df == 0).sum()

However, there a quite a significant amount of sector_IDs missing and also a lot of ratings.

### Unique values

A naive calculation on the number of unique sectors and routes within these sectors. Submitted by a number of distinct users.

In [ ]:
df[['sector_id', 'route', 'user_id']].nunique()

Why do we call the above estimate _naive_? 

According to [climb-europe.com](http://www.climb-europe.com/RockClimbingGermany/RockClimbingFrankenjura.html) _"there are approximately 1,000 crags spread out in a beautiful forest terrain."_ (Note that _crags_ in the above quote is the same as _sectors_ in our dataset.) This seems fine since in our dataset there are 351 different sectors noted.

However, let's look closer here. What are the records where `sector_id` is 0?  
We take a look at a number of samples.

In [ ]:
df[df['sector_id'] == 0].head(10)

So, what we can see is, that `sector_id == 0` actually does not belong to a proper sector, but rather denotes that sector is not known in this case. Later we'll have to remove those entries.

In the article it continues to claim that _Frankenjura boasts in excess of 10,000 routes._ Now this is where we should get a little suspicious. In only 350 sectors our dataset apparently contains already more than 12,000 routes - which is well above the 10,000 mentioned in the article.

Lets dig deeper here. Maybe there are a number of duplicates in the registered routes. Let's see if an example can confirm this.

In [ ]:
df[df['route'].str.startswith('knack')].drop_duplicates(subset=['route'])['route']

We will have to consolidate those duplicates during Preprocessing.

## Exploratory Visualization

### Missing values

Lets visualize the **Missing** and **Zero values** per column.

In [ ]:
plot_helper.missing_values_overview(df)

In our data set `0` is in indicator for missing values, except for `sex` where `0` indicates `male` (and `1` for `female`).

From the graph above we can conclude that we have missing data in `sector_id` and `rating`.

Since `sector_id` and `rating` are important for our analysis, we have to consider what to do about those missing values during Data Preparation later on.

## Analysis of our target variable `rating`

What are the unique values of our target variable?

In [ ]:
df_y = df['rating']
df_y.unique()

And how is their distribution?

In [ ]:
total = df_y.count()

ax = df_y.value_counts().sort_index().plot(
    kind='bar', title='ratings distribution')
_ = ax.set_ylabel('logged ascents')
_ = ax.set_xlabel('given rating')

for i in ax.patches:
    _ = ax.text(i.get_x()-.03, i.get_height()+.5, \
            str(round((i.get_height()/total)*100, 1))+'%')

From the distribution of the target variable `rating`, we can see that there are four distinct values. With 52.7% of ratings equal to 0, only about half of the ascents have been rated.

In the climbing community it is commonplace to rate only good climbs by marking them with a star. Thus, the values `1`, `2` or `3` correspond to one, two or three stars. Where one star is a good route and three stars is an exceptional good climb.

Hence the value `0`, or zero stars, denotes that a climb was either not rated or was not worth a star according to that used.

In [ ]:
df_y[df_y != 0].describe()

Out of those climbs, that were rated, the average is 2.14.

What is the percentage of users that rated at least one climb?

In [ ]:
(df.groupby('user_id')['rating'].sum() == 0).value_counts()

This means that approx. 2/3 of users have rated at least one item.

How many ratings are there per user?

In [ ]:
ratings_per_user = df.groupby('user_id')['rating'].apply(lambda x: x[x != 0].count())
ax = ratings_per_user.sort_values().plot(
    kind='line', title='No. of ratings per user (sorted by no. of ratings)', use_index=False)
_ = ax.set_ylabel('no. of ratings')
_ = ax.set_xlabel('user id')

From the above plot we can see that there must be one or several outliers with a large number of ratings (around 1600). Let's apply the logarithm on the y-axis to get a better picture.

In [ ]:
ax = ratings_per_user.sort_values().plot(
    kind='line', title='No. of ratings per user (logscale)', use_index=False, logy=True)
_ = ax.set_ylabel('no. of ratings')
_ = ax.set_xlabel('user id')

Now, this has more information. We can see that approx. 30% of users gave no rating at all. And more than 90% of users gave less than 200 ratings.

Let's look at the percentiles information in detail to confirm this.

In [ ]:
ratings_per_user.describe(percentiles=[.25, 0.33, .50, .75, .90, .95, .99])

## Algorithms and Techniques

## Benchmark

As a benchmark we'll use a __random recommender__, i.e. for a given user, any route will be given a random rating from 1, 2 or 3 stars. This will be our quantitative measurement to compare against.  
For a __qualitative benchmark__, we'll compare results to the recommendations by climbing magazine _klettern.de_.

# Methodology

## Data Preprocessing

Remove records from dataset where `sector_id == 0`

In [ ]:
df = df[df['sector_id'] != 0]

### Consolidate route names, remove duplicates

Since our data set does not provide a unique id for routes, we can only go by the name to identify a route. Unfortunately, there are many versions of route names as well as spelling mistakes. In order to arrive at a unique list of routes, we need to do some cleaning up.  

The following example illustrates that.

In [ ]:
df[df['route'].str.startswith('Adr')].drop_duplicates(subset=['route'])['route']

Let's start by converting all route names to lowercase and removing special characters.

In [ ]:
def string_cleaning(df, columns):
    '''Remove special characters and convert to lowercase. Apply to all columns as specified'''

    df[columns] = df[columns].applymap(lambda s: s.lower())

    # Need to use Series because regular replace() cannot handle RegEx
    for c in columns:
        df[c] = df[c].str.replace('[^A-Za-z\s]+', '') # TODO keep numbers??
        
        # Remove rows with empty values as they are unusable
        df = df[df[c] != '']

    return df

df = string_cleaning(df, ['route'])

Next, let's define a couple of functions that will help us to clean up duplicate route names within sectors.

In [ ]:
# http://www.coli.uni-saarland.de/courses/LT1/2011/slides/Python-Levenshtein.html
from Levenshtein import jaro_winkler

def create_jaro_winkler(keys, prefix_weight=None):
    '''Calculates Jaro-Winkler-Distance among all items of `keys` and returns calculation results as a matrix'''

    # TODO  is there a more pythonic way of calculating this matrix?
    
    jaro_winkler_matrix = np.ones((len(keys), len(keys)))

    i = 0
    while i <= len(keys) - 1:
        j = 0
        while j <= len(keys) - 1:
            if i != j:
                jaro_winkler_matrix[i][j] = jaro_winkler(
                    keys[i], keys[j]) if prefix_weight is None else jaro_winkler(keys[i], keys[j], prefix_weight)
            j += 1
        i += 1

    return jaro_winkler_matrix

def remove_overlap(d):
    '''Resolve 'paths' within the dictionary, e.g. 
     { A : B, B : C }  ->  { A : C, B : C }
     Returns cleaned dictionary'''

    set_keys = set(d.keys())
    set_values = set(d.values())
    set_intersect = set_keys.intersection(set_values)

    for v in set_intersect:
        new_target = d[v]

        filtered_dict = {key: new_target for (
            key, value) in d.items() if value == v}
        filtered_dict

        for i, j in d.items():
            if i in filtered_dict.keys():
                d[i] = filtered_dict[i]

    return d

In [ ]:
def get_routes_per_sector(df, sector_id):
    '''Returns a series with route names as labels and count of route name occurrences as values'''

    df_one_sector = df[df['sector_id'] == sector_id]
    return df_one_sector['route'].value_counts()

def clean_route_names_sector(df, sector_id, threshold=0.9, prefix_weight=1/100, debug=False):
    '''Replaces route name within given `sector_id` with the most similar matching name that has the most occurrences.'''

    routes_per_sector = get_routes_per_sector(df, sector_id)
    keys = routes_per_sector.keys()
    values = routes_per_sector.values

    # Create Jaro-Winkler-Matrix and fill diagonal with zeros, 
    # otherwise max value will always be 1.0 (as diagonal is always 1.0)
    jaro_winkler_matrix = create_jaro_winkler(keys, prefix_weight)
    np.fill_diagonal(jaro_winkler_matrix, 0)

    # empty nested dictionary, for replacements of similar route names
    replacements = {'route': {}}

    # find max count of values above threshold
    indices_above_threshold = np.transpose(
        np.nonzero(jaro_winkler_matrix >= threshold))

    first_values = [t[0] for t in indices_above_threshold]
    unique_first_values = np.unique(first_values)

    for u in unique_first_values:
        tuples_list = [t for t in indices_above_threshold if t[0] == u]
        counts_list = [values[t2[1]] for t2 in tuples_list]
        val_max = max(counts_list)
        index_max = counts_list.index(max(counts_list))
        keys[tuples_list[index_max][1]]

        if values[u] < values[tuples_list[index_max][1]]:
            replacements['route'][keys[u]] = keys[tuples_list[index_max][1]]
            if debug:
                print(
                    f'Replacement: {keys[u]}({values[u]}) : {keys[tuples_list[index_max][1]]}({values[tuples_list[index_max][1]]}) ')        

    # Resolve overlaps in replacements
    replacements['route'] = remove_overlap(replacements['route'])
        
    # Replace all found similar route names
    df.loc[df['sector_id'] == sector_id] = df.replace(
        to_replace=replacements)

    return df

def clean_route_names(df):
    '''Cleans up duplicate route names within sectors.'''
    
    for sid in df['sector_id'].unique():
        df = clean_route_names_sector(df, sid)
    return df   

In [ ]:
df = clean_route_names(df)

In [ ]:
# jaro_winkler('adrschpach weg', 'adrspach weg', 1/100)
df[df['route'].str.startswith('adr')].drop_duplicates()

### Get list of unique routes

Extract all the individual routes. We also keep `sector_id` here, because there can be routes with the same name that lie in different sectors.

In [ ]:
df_routes = df[['sector_id', 'route']].drop_duplicates(['sector_id', 'route'])
a = df_routes.shape[0]
md(f'So, our new count for routes is **{a}**, which is a lot less than what our naive count was.')

Reset the index of our consolidated list of routes.

In [ ]:
df_routes.index.name = 'route_id'
df_routes.reset_index(inplace=True)
df_routes.head(5)

### Get list of unique users

In [ ]:
df_users = df['user_id'].drop_duplicates()
df_users.reset_index(inplace=True, drop=True)
df_users.head(5)

### Get list of ratings

In [ ]:
df_ratings = pd.merge(df_routes, df, on=['sector_id', 'route'])[['route_id', 'sector', 'route', 'user_id', 'rating']]
df_ratings['sector_route'] = df_ratings['sector'] + '_' + df_ratings['route']
df_ratings.head(5)

In [ ]:
# Filter ratings, only include non-NaN values
df_ratings = df_ratings[df_ratings['rating'] != 0]

## Implementation

### Implement random model 

In [ ]:
from random import randint

df_ratings_random = df_ratings
df_ratings_random['rating_random'] = df_ratings_random.apply(lambda row: randint(1, 3), axis=1)

df_ratings_random.head(5)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = df_ratings_random['rating']
y_predict = df_ratings_random['rating_random']

rms = sqrt(mean_squared_error(y_actual, y_predict))
f'The RMSE for our random model is {rms:.4f}'

Tbe benchmark of our random model is `RMSE = 1.1321`, which is the value we want to _beat_ with our recommender model.

### Implement K-Means model

In [ ]:
user_route_ratings = pd.pivot_table(df_ratings, index='user_id', columns= 'sector_route', values='rating')

print('dataset dimensions: ', user_route_ratings.shape, '\n\nSubset example:')
user_route_ratings.iloc[:6, :10]

In [ ]:
# user_id = 39
# # user_id = 35939
# X_test[X_test['user_id'] == user_id]
# # user_route_ratings.loc[user_id]

In [ ]:
def get_most_rated_routes(user_route_ratings, max_number_of_routes):
    # 1- Count
    user_route_ratings = user_route_ratings.append(
        user_route_ratings.count(), ignore_index=True)
    # 2- sort
    user_route_ratings_sorted = user_route_ratings.sort_values(
        len(user_route_ratings)-1, axis=1, ascending=False)
    user_route_ratings_sorted = user_route_ratings_sorted.drop(
        user_route_ratings_sorted.tail(1).index)
    # 3- slice
    most_rated_routes = user_route_ratings_sorted.iloc[:,
                                                       :max_number_of_routes]
    return most_rated_routes


def get_users_who_rate_the_most(most_rated_routes, max_number_of_routes):
    # Get most voting users
    # 1- Count
    most_rated_routes['counts'] = pd.Series(most_rated_routes.count(axis=1))
    # 2- Sort
    most_rated_routes_users = most_rated_routes.sort_values(
        'counts', ascending=False)
    # 3- Slice
    most_rated_routes_users_selection = most_rated_routes_users.iloc[:max_number_of_routes, :]
    most_rated_routes_users_selection = most_rated_routes_users_selection.drop([
                                                                               'counts'], axis=1)

    return most_rated_routes_users_selection


def sort_by_rating_density(user_route_ratings, n_routes, n_users):
    most_rated_routes = get_most_rated_routes(user_route_ratings, n_routes)
    most_rated_routes = get_users_who_rate_the_most(most_rated_routes, n_users)
    return most_rated_routes

In [ ]:
n_routes = 30
n_users = 18
most_rated_routes_users_selection = sort_by_rating_density(user_route_ratings, n_routes, n_users)

print('dataset dimensions: ', most_rated_routes_users_selection.shape)
most_rated_routes_users_selection.head()

In the above table `sector_route` is label of column names, not the label of the first column. cp. to first table in Implementation section

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def draw_routes_heatmap(most_rated_routes_users_selection, axis_labels=True):

    # Reverse to match the order of the printed dataframe
    #most_rated_routes_users_selection = most_rated_routes_users_selection.iloc[::-1]

    fig = plt.figure(figsize=(15, 4))
    ax = plt.gca()

    # Draw heatmap
    heatmap = ax.imshow(most_rated_routes_users_selection,
                        interpolation='nearest', vmin=0, vmax=3, aspect='auto')

    if axis_labels:
        ax.set_yticks(
            np.arange(most_rated_routes_users_selection.shape[0]), minor=False)
        ax.set_xticks(
            np.arange(most_rated_routes_users_selection.shape[1]), minor=False)
        ax.invert_yaxis()
        ax.xaxis.tick_top()
        labels = most_rated_routes_users_selection.columns.str[:40]
        ax.set_xticklabels(labels, minor=False)
        ax.set_yticklabels(
            most_rated_routes_users_selection.index, minor=False)
        plt.setp(ax.get_xticklabels(), rotation=90)
    else:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    ax.grid(False)
    ax.set_ylabel('User id')

    # Separate heatmap from color bar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Color bar
    cbar = fig.colorbar(heatmap, ticks=[3, 2, 1, 0], cax=cax)
    cbar.ax.set_yticklabels(['3 stars', '2 stars', '1 stars', '0 stars'])

    plt.show()

In [ ]:
draw_routes_heatmap(most_rated_routes_users_selection)

In [ ]:
sparse_ratings = csr_matrix(pd.SparseDataFrame(user_route_ratings).to_coo())

### Clustering

In [ ]:
from sklearn.cluster import KMeans

def clustering_errors(k, data):
    kmeans = KMeans(n_clusters=k).fit(data)
    predictions = kmeans.predict(data)
    silhouette_avg = silhouette_score(data, predictions)
    return silhouette_avg

In [ ]:
# Choose the range of k values to test.
# We added a stride of 5 to improve performance. We don't need to calculate the error for every k value
# possible_k_values = range(2, len(X)-1, 5)
possible_k_values = range(2, 20, 2)

# Calculate error values for all k values we're interested in
errors_per_k = [clustering_errors(k, sparse_ratings) for k in possible_k_values]


In [ ]:
# Plot each value of K vs. the silhouette score at that value
fig, ax = plt.subplots(figsize=(16, 6))
_ = ax.set_xlabel('K - number of clusters')
_ = ax.set_ylabel('Silhouette Score (higher is better)')
_ = ax.plot(possible_k_values, errors_per_k)

# Ticks and grid
xticks = np.arange(min(possible_k_values), max(possible_k_values)+1, 5.0)
_ = ax.set_xticks(xticks, minor=False)
_ = ax.set_xticks(xticks, minor=True)
_ = ax.xaxis.grid(True, which='both')
yticks = np.arange(round(min(errors_per_k), 2), max(errors_per_k), .05)
_ = ax.set_yticks(yticks, minor=False)
_ = ax.set_yticks(yticks, minor=True)
_ = ax.yaxis.grid(True, which='both')

In [ ]:
# Predict user clusters using KMeans
predictions = KMeans(n_clusters=2, algorithm='full').fit_predict(sparse_ratings)

In [ ]:
len(predictions)

In [ ]:
unique, counts = np.unique(predictions, return_counts=True)
dict(zip(unique, counts))

In [ ]:
def draw_route_clusters(clustered, max_users, max_routes):
    c = 1
    for cluster_id in clustered.group.unique():
        # To improve visibility, we're showing at most max_users users and max_routes routes per cluster.
        # You can change these values to see more users & routes per cluster
        d = clustered[clustered.group == cluster_id].drop(
            ['user_id', 'group'], axis=1)
        n_users_in_cluster = d.shape[0]

        d = sort_by_rating_density(d, max_routes, max_users)

#         d = d.reindex_axis(d.mean().sort_values(ascending=False).index, axis=1)
#         d = d.reindex_axis(d.count(axis=1).sort_values(ascending=False).index)
        d = d.reindex(d.mean().sort_values(ascending=False).index, axis=1)
        d = d.reindex(d.count(axis=1).sort_values(ascending=False).index)
        d = d.iloc[:max_users, :max_routes]
        n_users_in_plot = d.shape[0]

        # We're only selecting to show clusters that have more than 9 users, otherwise, they're less interesting
        if len(d) > 9:
            print('cluster # {}'.format(cluster_id))
            print('# of users in cluster: {}.'.format(n_users_in_cluster),
                  '# of users in plot: {}'.format(n_users_in_plot))
            fig = plt.figure(figsize=(15, 4))
            ax = plt.gca()

            ax.invert_yaxis()
            ax.xaxis.tick_top()
            labels = d.columns.str[:40]

            ax.set_yticks(np.arange(d.shape[0]), minor=False)
            ax.set_xticks(np.arange(d.shape[1]), minor=False)

            ax.set_xticklabels(labels, minor=False)

            ax.get_yaxis().set_visible(False)

            # Heatmap
            heatmap = plt.imshow(d, vmin=1, vmax=3, aspect='auto')

            ax.set_xlabel('routes')
            ax.set_ylabel('User id')

            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.05)

            # Color bar
            cbar = fig.colorbar(heatmap, ticks=[3, 2, 1], cax=cax)
            cbar.ax.set_yticklabels(
                ['3 stars', '2 stars', '1 stars'])

            plt.setp(ax.get_xticklabels(), rotation=90, fontsize=9)
            plt.tick_params(axis='both', which='both', bottom='off',
                            top='off', left='off', labelbottom='off', labelleft='off')
            # print('cluster # {} \n(Showing at most {} users and {} routes)'.format(cluster_id, max_users, max_routes))

            plt.show()

In [ ]:
max_users = 70
max_routes = 50

# is this line messing up my user_id?
clustered = pd.concat([user_route_ratings.reset_index(),
                       pd.DataFrame({'group': predictions})], axis=1)
clustered = clustered.set_index('user_id')
clustered['user_id'] = clustered.index

In [ ]:
# Get a random cluster number with cluster size >= threshold
threshold = 1
group_counts = clustered['group'].value_counts()
group_counts = group_counts[group_counts >= threshold]
group_counts
random_cluster_number = group_counts.sample(1).index[0]

In [ ]:
draw_route_clusters(clustered, max_users, max_routes)


## Prediction
Let's pick a cluster and a specific user and see what useful things this clustering will allow us to do.

Let's first pick a cluster:

In [ ]:
def get_cluster(clustered, cluster_number):
    
#     cluster = clustered[clustered.group == cluster_number].drop(['user_id', 'group'], axis=1)
    cluster = clustered[clustered.group == cluster_number].drop(['group'], axis=1)
    cluster = cluster.set_index('user_id')
    cluster = cluster.rename_axis('user_id')
    
    return cluster

In [ ]:
# TODO: Pick a cluster ID from the clusters above
cluster_number = random_cluster_number

# Let's filter to only see the region of the dataset with the most number of values 
n_users = 75
n_routes = 300
cluster = get_cluster(clustered, cluster_number)

cluster = sort_by_rating_density(cluster, n_routes, n_users)
draw_routes_heatmap(cluster, axis_labels=False)

# set cluster back to original (because sort_by_rating_density() modifies cluster)
cluster = get_cluster(clustered, cluster_number) 

In [ ]:
route_name = 'Wolfsberger Grotte_panda'

cluster[route_name].mean()

In [ ]:
def cluster_membership(clustered, user_id):
    '''given a clustered dataframe, returns the cluster number that user_id is a member of'''
    
    return clustered[clustered['user_id'] == user_id]['group'].iloc[0]

In [ ]:
%%latex

Weighted Rating (WR) = $(\frac{v}{v + m} \cdot R) + (\frac{m}{v + m} \cdot C)$

where
- *v* is the number of ratings for that route
- *m* is the minimum ratings required to be listed in the chart
- *R* is the average rating of the route
- *C* is the mean vote across the whole report

In [ ]:
def cluster_avg_count(cluster):
    '''Given a cluster, calculate each route's avg rating and count of ratings. '''
    
    df = pd.DataFrame()
    df['sector_route'] = cluster.columns

    for i, row in df.iterrows():
        rating_average = cluster[row['sector_route']].mean()
        rating_count = cluster[row['sector_route']].count()
        df.at[i,'rating_average'] = rating_average
        df.at[i,'rating_count'] = rating_count
        
    return df

In [ ]:
def top_n_routes_cluster(cluster, n=0):
    '''Calculate weighted rating of all routes. Return top-n routes. If n==0, return all routes'''

    percentile = 0.85

    # get count and mean of ratings per route for this cluster
    df_counts_average = cluster_avg_count(cluster)

    C = df_counts_average['rating_average'].mean()
    m = df_counts_average['rating_count'].quantile(percentile)

    # Filter for routes that have ratings above threshold m
    qualified = df_counts_average[(df_counts_average['rating_count'] >= m) & (
        df_counts_average['rating_count'].notnull()) & (df_counts_average['rating_average'].notnull())]

    # calculate weighted rating for each route
    col_wr = qualified.apply(lambda x: (x['rating_count']/(x['rating_count']+m)
                                                 * x['rating_average']) + (m/(m+x['rating_count']) * C), axis=1)
    qualified = qualified.assign(wr=col_wr.values)
    
    # Sort by rating
    qualified = qualified.sort_values(
        'wr', ascending=False) if n == 0 else qualified.sort_values('wr', ascending=False).head(n)

    # Return the top n routes
    return qualified

def top_n_routes_user(clustered, user_id, n):
    '''Return top n routes of a user'''

    # get cluster number
    cluster_number = cluster_membership(clustered, user_id)

    # retrieve cluster by cluster number
    cluster = get_cluster(clustered, cluster_number)

    # retrieve top routes by cluster
    df_top_routes_cluster = top_n_routes_cluster(cluster)

    # subtract user's already climbed and rated routes from result
    df_top_routes_cluster_without_user = df_top_routes_cluster.drop(
        df_top_routes_cluster[df_top_routes_cluster['sector_route'].isin(cluster.loc[user_id].dropna().index)].index)

    return df_top_routes_cluster_without_user.head(n)

In [ ]:
user_id = 32
clustered[clustered['user_id'] == user_id]

In [ ]:
user_id = 32

cluster_number = cluster_membership(clustered, user_id)
f'cluster_number: {cluster_number}'
cluster = get_cluster(clustered, cluster_number)
cluster

top_n = top_n_routes_cluster(cluster)
top_n.shape

top_n_without_user = top_n.drop(
    top_n[top_n['sector_route'].isin(cluster.loc[user_id].dropna().index)].index)

top_n_without_user.shape

In [ ]:
cluster = get_cluster(clustered, random_cluster_number)
cluster.tail(10)

### Qualitative result check

For a qualitative result check, let's look at the top 20 recommended routes within a cluster and count how many of the routes show up in the Top 100 list of climbing magazine _klettern_. The list is available on the [klettern.de](https://www.klettern.de/sixcms/media.php/8/Top100-Kletterrouten_Frankenjura.pdf) website.

In [ ]:
f'Top 20 Cluster'
top_n_routes_cluster(cluster, 20)

We find 9 out of 20 recommended routes also in the Top 100 list. So, our recommendations seem to be alright.

<img src="images/qualitative_benchmark.png" alt="Qualitative Benchmark" width="500" align="left" />

The suggested routes are all in the upper experienced to expert level. Which is no surprise, as we have seen earlier, that the majority of climbed routes is from that range.

In [ ]:
users = [32, 493, 35939, 63837]
for u in users:
    f'user_id: {u} is member of cluster number: {cluster_membership(clustered, u)}'
    top_n_routes_user(clustered, u, 5)
    f''

In [ ]:
def predict_user_rating(user_id, sector_route):
    '''Predict what a user's rating for a particular route would be.'''
    
    # determine cluster membership and get cluster
    cluster_number = cluster_membership(clustered, user_id)
    cluster = get_cluster(clustered, cluster_number)
    
    # get ratings<->routes for cluster
    rated_routes = top_n_routes_cluster(cluster)
    
    # get rating of route
    rating = rated_routes[rated_routes['sector_route'] == sector_route]['wr']
    
    return rating.iloc[0]

In [ ]:
predict_user_rating(32, 'Eldorado_ekel')

### Alternative Implementation -  SVD

#### Trying Surprise package

In [ ]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 3))

data = Dataset.load_from_df(df_ratings[['user_id', 'sector_route', 'rating']], reader)

In [ ]:
svd = SVD()
_ = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

Following the example provided on the website of the [surprise package](https://surprise.readthedocs.io/en/stable/FAQ.html), we can implement a top n recommender like this.

In [ ]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

In [ ]:
import gc
gc.disable()  # following recommendation of https://bugs.python.org/issue2607

# Print the recommended routes for each user
counter = 0
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print()
    counter += 1
    if counter == 3:
        break
        
gc.enable()

In [ ]:
# import gc
gc.disable()  # following recommendation of https://bugs.python.org/issue2607
top_n[5512]
gc.enable()

## Refinement

#### Tuning , GridSearch

Let's run a grid search on a number of parameters to see if any improvement of our model is possible.

In [ ]:
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.007],
              'reg_all': [0.2, 0.4, 0.6]}

In [ ]:
from surprise.model_selection import GridSearchCV

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
svd_refined = SVD(n_epochs=15, lr_all=0.007, reg_all=0.2)
_ = cross_validate(svd_refined, data, measures=['RMSE'], cv=5, verbose=True)

# Results

## Model Evaluation and Validation

In our final solution we were able to give an individual recommendation of the top n routes for any individual user. This solves the problem as stated earlier.  

Our final model uses `SVD` to produce recommendations based solely on user ratings. This approach was chosen as the previous attempt, using K-Means, failed to produce a significant cluster diversification (more than 99% of users were ended up in the same cluster). SVD has proven very useful in recommendation tasks, e.g. Netflix movie recommendation.  

##### Assessing the robustness of our algorithm by comparing results of explicit vs implicit feedback.

The SVD algorithm considers only __explicit feedback__. I.e. if a user gives a rating of one star, then this is counted as a one star rating. However, the __implicit feedback__ of a user is not considered. Implicit feedback means that a user, by chosing to rate a particular route he or she climbed (as opposed to not rating that route) already provides a positive indication for that route.

The __SVD++__ algorithm can take that implicit feedback into account.

In [ ]:
svd_pp = SVDpp(n_epochs=15, lr_all=0.007, reg_all=0.2)
_ = cross_validate(svd_pp, data, measures=['RMSE'], cv=5, verbose=True)

In [ ]:
import matplotlib.pyplot as plt

algos = ('Random recomm.', 'SVD', 'SVD++')
y_pos = np.arange(len(algos))
rmse = [1.1321, 0.6503, 0.6475]

_ = plt.bar(y_pos, rmse, align='center', alpha=0.5)
_ = plt.xticks(y_pos, algos)
_ = plt.ylabel('RMSE')
_ = plt.title('Comparing RMSE')

plt.show()

Thus we can see that there is no significant difference between implicit and explicit feedback.

## Justification

The comparison of our solution to the initial benchmark:

|Model|Random model|SVD   |SVD tuned|
|-----|:----------:|:----:|:-------:|
|RMSE |1.1321      |0.6510|0.6489   |

We can see a significant improvement of _RMSE_ from Random model over the SVD to the tuned SVD model (smaller RMSE is better), although the tuning did not gain much.

Thus we have solved the problem of personalised recommendations for climbers.


# Conclusion

## Free-Form Visualization

Let's look at the distribution of routes by climbing grade (= difficulty level of the route) to understand whether our climbers are mostly beginners, advanced or professionals.

In [ ]:
_ = df_original.groupby('grade').count()['id'].plot(kind='bar', color='b', title='ascents by grade')

Our distribution seems to have the general form of a bell shape, i.e. a Gaussian distribution. What seems odd though, are the two dents in the distribution at 6c and 7b.

For this to understand we need to have some background knowledge on the different climbing grading systems used in different countries. The grading system used by 8a.nu, which is the source of our data set, is the _French_ grading system. In Germany a different grading system is used called _UIAA_.

One peculiar thing about different grading systems is, that they do not follow the same step size. E.g. in French scale grading the step from one grade to the next higher grade could be smaller than the steps in between grades in UIAA. This can be seen when looking at a grade comparison chart as shown below.

<img src="images/climbing_grades_comparison.png" alt="Grade comparison table" width="500" align="left" />

All the routes in Germany are given grades following UIAA scale. If a climber wants to log a climb in Germany with a UIAA grade on 8a.nu, he has to convert the grade to French scale. As there are no real corresponding entries for 6c and 7b in the UIAA scale, this explains the dent in our distribution.

Therefore, joining the numbers from 6c and 6b+, as well as 7a+ and 7b seems reasonable. 

In [ ]:
df_grade_adjusted = df_original.copy()
df_grade_adjusted.loc[df_grade_adjusted['grade'] == '6c'] = '6b+'
df_grade_adjusted.loc[df_grade_adjusted['grade'] == '7b'] = '7a+'
_ = df_grade_adjusted.groupby('grade').count()['id'].plot(
    kind='bar', color='b', title='ascents by grade (adjusted)')

And the result of the distribution of logged climbs is now as expected.

With the majority of climbs around 6c+, 7a level, what does this actually mean? Let's take a look at the assessment given by the website [thecrag.com](https://www.thecrag.com/en/article/grades#grade-ranges). And we can see that the majority of logged ascents are in the upper _Experienced_ range.

<img src="images/climbing_levels.png" alt="Climbing levels" width="500" align="left" />

## Reflection

## Improvement